# Arquivo de coleta de dados do BenchMark

## Atualizando o path do sistema

In [ ]:
import os
import sys

current_dir: str = os.path.abspath("../../..")
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)
print(sys.path)


## Importando as bibliotecas internas

In [ ]:
from src.python.libs.benchmark.data import (BenchMetrics,
                                            FIRST_ORDER, SECOND_ORDER, THIRD_ORDER, FOURTH_ORDER, 
                                            FIRST_SIZE, SECOND_SIZE, THIRD_SIZE, FOURTH_SIZE,
                                            BUBBLE_NAME, MERGE_NAME, QUICK_NAME)

from src.python.libs.order_algorithms.bubble_sort import BubbleSort
from src.python.libs.order_algorithms.merge_sort import MergeSort
from src.python.libs.order_algorithms.quick_sort import QuickSort

from src.python.libs.graphics.graphic_generator import GraphicData, GraphicGenerator

## Importando as bibliotecas externas

In [ ]:
from typing import Union
import pandas as pd
from random import randint

## Definindo algumas constantes

In [ ]:
PATH: str = "../../results/data"

## Criando uma função específica para criar dados para o quicksort

In [ ]:
REPEATER_FACTOR: int = 100

MIN_NUMBER: int = 0
MAX_NUMBER: int = 1_000_000
SIZES: dict[int, int] = {
            1: FIRST_SIZE,
            2: SECOND_SIZE,
            3: THIRD_SIZE,
            4: FOURTH_SIZE
        }

def data_generator_quicksort(size: int, data_type):
    if data_type == FIRST_ORDER:
        numbers_list = [i for i in range(0, SIZES[size])]

    elif data_type == SECOND_ORDER:
        numbers_list = [i for i in range(SIZES[size]-1, -1, -1)]
    
    elif data_type == THIRD_ORDER:
        numbers_list = [randint(MIN_NUMBER, MAX_NUMBER) for n in range(SIZES[size])]
    
    elif data_type == FOURTH_ORDER:
        numbers_list = [randint(MIN_NUMBER, MAX_NUMBER) % REPEATER_FACTOR for c in range(SIZES[size])]

    return numbers_list

## Criando a função para rodar as simulações

In [ ]:
def simulation(algorithms: list[str]) -> GraphicData:
    graphic_data: GraphicData = GraphicData()

    for algorithm_name in algorithms:

        for size in range(1, 5):

            for order in [FIRST_ORDER, SECOND_ORDER, THIRD_ORDER, FOURTH_ORDER]:

                metrics: BenchMetrics = BenchMetrics() # Generate BenchMetrics

                if algorithm_name == BUBBLE_NAME:
                    algorithm = BubbleSort()
                    algorithm.data_generator(size, order)

                elif algorithm_name == MERGE_NAME:
                    algorithm = MergeSort()
                    algorithm.data_generator(size, order)
                    
                elif algorithm_name == QUICK_NAME:
                    # Informação: O python exige que criemos a lista fora da classe 
                    # para não ocorrer erro de denpth recursion. Provavelmente é 
                    # problema do compilador, pois no Merge sort funciona normalmente 
                    # e criando fora da classe funciona. Enfim, pythonices.
                    algorithm = QuickSort()
                    algorithm.data_type = order
                    algorithm.numbers_list = data_generator_quicksort(size, order) 
                
                elif ((algorithm_name == BUBBLE_NAME and size == FOURTH_SIZE) or 
                      (algorithm_name == QUICK_NAME and size == FOURTH_SIZE)):
                    # Pela quantidade muito alta de tempo que leva nesse algoritmos,
                    # torna-se inviável roda-los utilizando QuickSort e BubbleSort,
                    # pois, no pior dos casos, ambos tem complexidade de O(n^2)
                    continue

                metrics = algorithm.benchmarkSort(metrics)
                print(f"""
                -------------------------------------------
                Algoritmo: {metrics.algorithm_name}
                Ordem: {metrics.data_type}
                Tamanho: {metrics.list_size}
                Tempo: {metrics.execution_time}
                Memoria: {metrics.memory_usage[1] - metrics.memory_usage[0]}
                Comparações: {metrics.comparations}
                Trocas: {metrics.swaps}
                -------------------------------------------
                """)
                graphic_data.insert_data(metrics)
        graphic_data.save_as_csv(PATH)

    return graphic_data

## Rodando as simulações

In [ ]:
algorithms: list[str] = [MERGE_NAME, QUICK_NAME, BUBBLE_NAME]
graphic_data: GraphicData = simulation(algorithms=algorithms)

## Mostrando os resultados das simulações

In [ ]:
print(graphic_data)

## Salvando as informações como um arquivo csv

In [ ]:
graphic_data.save_as_csv(PATH)